In [ ]:
!pip install pororo -q
!pip install emoji -q

     |████████████████████████████████| 256 kB 13.8 MB/s 
     |████████████████████████████████| 3.5 MB 47.7 MB/s 
     |████████████████████████████████| 5.9 MB 43.8 MB/s 
     |████████████████████████████████| 748.8 MB 21 kB/s 
     |████████████████████████████████| 3.1 MB 47.1 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 1.5 MB 41.9 MB/s 
     |████████████████████████████████| 42.4 MB 1.3 MB/s 
     |████████████████████████████████| 468 kB 61.1 MB/s 
     |████████████████████████████████| 1.7 MB 57.4 MB/s 
     |████████████████████████████████| 1.2 MB 58.8 MB/s 
     |████████████████████████████████| 145 kB 64.3 MB/s 
     |████████████████████████████████| 90 kB 10.1 MB/s 
     |████████████████████████████████| 748 kB 74.9 MB/s 
     |████████████████████████████████| 1.2 MB 73.7 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 69.5 MB/s 
     |█████████████

In [ ]:
from pororo import Pororo
# import tensorflow as tf
review = Pororo(task="review", lang="ko")


In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

from datetime import datetime, timedelta
import time 

In [2]:
start_dt = datetime.today() - timedelta(0)    # 현재 in google
start_dt = start_dt.strftime('%Y-%m-%d')
start_dt = datetime.strptime(start_dt,'%Y-%m-%d')

In [ ]:
date_start = (datetime.today()- timedelta(30)).strftime('%Y-%m-%d %H:%M:%S')
date_end = (datetime.today()- timedelta(0)).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
date_start, date_end

('2022-01-19 01:59:58', '2022-02-18 01:59:58')

In [3]:
url        = 'https://api.prizm.co.kr/v1/showroom/21212/deals?size=20'
review_url = ''

In [4]:
 headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
 PRIZM = requests.get(url, headers=headers).json()

In [24]:
i = 3
PRODUCT = PRIZM['content'][i]['goods']['name']
code = PRIZM['content'][i]['goods']['code']
URL = 'https://mweb.prizm.co.kr/goods/' + str(code)
PRODUCT, URL

('어메이징 오트 바리스타 950ml*3', 'https://mweb.prizm.co.kr/goods/iwvyibu')

In [ ]:
# 수집대상 : List 형식
review_date = []
review_user = []
review_text = []
review_gubun= []
review_prodt= []
review_score= []

date_start = (datetime.today()- timedelta(90)).strftime('%Y-%m-%d %H:%M:%S')
date_end = (datetime.today()- timedelta(0)).strftime('%Y-%m-%d %H:%M:%S')

def wadiz_review(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.141 Safari/537.36'}
    wadiz = requests.get(url, headers=headers).json()
    for i in range(0, len(PRIZM['content']) ):
        REVIEW_DT = wadiz['data']['content'][i]['whenCreated']
        if date_start <=  REVIEW_DT  <= date_end :
            # 제품정보 수집
            NAME = wadiz['data']['content'][i]['nickName']
            PRODUCT = PRIZM['content'][i]['goods']['name']
            REVIEW = wadiz['data']['content'][i]['body'].replace('\n', ' ')
            REVIEW_GUBUN = wadiz['data']['content'][i]['commentType']
            try:
                SCORE = review(REVIEW)
            except:
                SCORE = 0
            review_score.append(SCORE)
            review_prodt.append(PRODUCT)
            review_date.append(REVIEW_DT)
            review_user.append(NAME)
            review_text.append(REVIEW)
            review_gubun.append(REVIEW_GUBUN)
        else:
            pass

def get_wadiz_review(t1):
    url_list = [
            'https://www.wadiz.kr/web/reward/api/comments/campaigns/135610?&commentGroupType=CAMPAIGN&page=0&size=100',
            # 'https://www.wadiz.kr/web/reward/api/comments/campaigns/135610?&commentGroupType=CAMPAIGN&page=1&size=100',
          ]
    for url in url_list:
        wadiz_review(url)

In [ ]:
get_wadiz_review('')

In [ ]:
result = pd.DataFrame()
result['DATE'] = review_date
result['NAME'] = review_user
result['REVIEW'] = review_text
result['REVIEW_GUBUN'] = review_gubun
result['PRODUCT'] = review_prodt
result['SCORE'] = review_score

In [ ]:
result.tail(20)

,DATE,NAME,REVIEW,REVIEW_GUBUN,PRODUCT,SCORE
32,2022-01-04 17:51:41.0,김성만,회사직원 생일선물이라 미리 알람신청하고 진행을 했는데 생각보다 인기 엄청 많네요. ...,SUPPORT,135610,4.95
33,2022-01-04 17:51:09.0,김희수,https://www.facebook.com/100039398194465/posts...,SUGGESTION,135610,4.84
34,2022-01-04 17:50:50.0,하얀♡,https://m.facebook.com/story.php?story_fbid=48...,SUPPORT,135610,5.01
35,2022-01-04 17:47:20.0,캬으힝,https://www.facebook.com/100038092123157/posts...,SUGGESTION,135610,3.83
36,2022-01-04 17:44:48.0,김민정,뤂덕으로써 놓칠수없습니다 루피만세!!!!!!!!,SUPPORT,135610,4.99
37,2022-01-04 17:29:12.0,지훈,"이미 키보드,마우스가 있지만 공간와디즈에서 피규어 실물영접하고 바로 펀딩했습니다.....",REVIEW,135610,4.88
38,2022-01-04 16:01:25.0,HAN,폰을 올려놓고 쓸 수 있는 부분에서 뤂덕은 이미 반해버렸어요. 잔망루피는 제가 데려...,SUPPORT,135610,4.94
39,2022-01-04 15:59:44.0,이인균,"너무 심하게 귀여운데, 세트 가격까지 합리적입니다. 앞으로 흥을 가지고 일할 수 있...",REVIEW,135610,5.02
40,2022-01-04 15:39:47.0,euna lee,저는 루피 팬도 아니었고 분홍색은 싫어합니다....... 하지만 공간에서 보고 홀린...,REVIEW,135610,4.81
41,2022-01-04 15:27:36.0,Rangny,솔직히.. 공간와디즈에서 보고 삘 받았어요.. 잔망루피 피규어.. 이건 내 꺼라고....,REVIEW,135610,4.57
